In [31]:
import pandas as pd
import numpy as np
import requests
import re
import math

import ssl
import urllib.request as request
from concurrent.futures import ThreadPoolExecutor as Executor
from requests import Session
from lxml import html
import pandas as pd
from datetime import datetime
import time
import random

import scattertext as st

nlp = st.WhitespaceNLP.whitespace_nlp

In [40]:
import logging
import psycopg2
from psycopg2 import sql

rds_host  = ""
name = "" 
db_name = "" 
password = ""


logger = logging.getLogger()
logger.setLevel(logging.INFO)

try:
    conn = psycopg2.connect(host=rds_host, user=name, password=password, database=db_name, port=5432, connect_timeout=5)
except psycopg2.OperationalError as e:
    logger.error("ERROR: Unexpected error: Could not connect to postGreSQL instance.")
    logger.error(e)
    sys.exit()

logger.info("SUCCESS: Connection to RDS postGreSQL instance succeeded")


def getLatestReviews(business_id, 
                     limit=200):
    sql = f'''
    SELECT date, 
           text,
           stars::INTEGER
    FROM tallyds.yelp_review
    WHERE business_id = '{business_id}'
    ORDER BY datetime DESC
    LIMIT {limit};
    '''
    with conn.cursor() as cursor:
        cursor.execute(sql)
        # return a list of tuples
        return cursor.fetchall()
    
data = getLatestReviews("jga_2HO_j4I7tSYf5cCEnQ")
df_reviews = pd.DataFrame(data, columns=['date', 'text', 'stars'])
df_reviews = df_reviews.head(200)

In [43]:
df_reviews['stars'].dtype

dtype('int64')

In [37]:
def getPosNegLongPhrases(df_reviews, topk=10):
#     df = pd.DataFrame(results)
#     df = df.rename(columns={0: 'date', 1: 'stars', 2: 'reviewText', 3: 'review_id', 4: 'user_id'})
    df = df_reviews 
    df['stars'] = df['stars'].astype(str)
    df = df.dropna()
    df['only_alphabets'] = df['text'].apply(lambda x: ' '.join(re.findall("[a-zA-Z]+", x)))

    replace_dict_phrase_count = {'[':'',']':'','-':'','!':'','.':'',"'":''}
    for key in replace_dict_phrase_count.keys():
        df['only_alphabets'] = df['only_alphabets'].str.replace(key, replace_dict_phrase_count[key])
        df['only_alphabets'] = df['only_alphabets'].str.lower()

    stopwords = ['"','+','@','&','*','\\',')','(','\(','\xa0','0','1','2','3','4','5','6','7','8','9','/','$',"'d","'ll","'m",'+','maybe','from','first','here','only','put','where','got','sure','definitely','food','yet','our','go','since','really','very','two',"n't",'with','if',"'s",'which','came','all','me','(',')','makes','make','were','immediately','get','been','ahead','also','that','one','have','see','what','to','we','had','.',"'re",'it','or','he','she','we','us','how','went','no','"','of','has','by','bit','thing','place','so','ok','and','they','none','was','you',"'ve",'did','be','and','but','is','as','&','you','has','-',':','and','had','was','him','so','my','did','would','her','him','it','is','by','bit','thing','place','[',']','while','check-in','=','= =','want', 'good','husband', 'want','love','something','your','they','your','cuz','him',"i've",'her','told', 'check', 'i"m', "it's",'they', 'this','its','they','this',"don't",'the',',', 'it', 'i"ve', 'i"m', '!', '1','2','3','4', '5','6','7','8','9','0','/','.']
    def filter_stopwords(text):
        for i in str(text):
            if i not in stopwords:
                return str(text)

    #if item in stopwords list partially matches, delete, single letters like 'i' would be deleted from inside individual words if in list
    df = df[~df['only_alphabets'].isin(stopwords)]
    # if the following words fully matches, filter out
    full_match_list = ['i','a','an','am','at','are','in','on','for','','\xa0\xa0','\xa0','\(']
    df = df[~df['only_alphabets'].isin(full_match_list)]

    corpus = st.CorpusFromPandas(df,
                                 category_col='stars',
                                 text_col='only_alphabets',
                                 nlp=nlp).build()
    term_freq_df = corpus.get_term_freq_df()
    term_freq_df = pd.DataFrame(term_freq_df.to_records())#flatten multi-level index to rename columns
    term_freq_df = term_freq_df.rename(columns = {'5 freq': '5.0', '4 freq': '4.0','2 freq': '2.0', '1 freq': '1.0' })

    categories = df['stars'].unique()
    high = np.array([])
    if '5' in categories:
        high = corpus.get_scaled_f_scores('5')
    elif '4' in categories:
        high = corpus.get_scaled_f_scores('4')
    if '1'  in categories:
        high =  corpus.get_scaled_f_scores('1')
    elif '2' in categories:
        high = corpus.get_scaled_f_scores('2')

    df_wordFreq = pd.DataFrame()
    columns = ['term', 'score']
    if high.shape[0] > 0:
        df_wordFreq = pd.DataFrame([term_freq_df.term.tolist(), high]).T
        df_wordFreq = df_wordFreq.sort_values(1, ascending=True)#.head(topk)
        df_wordFreq.columns = columns

    x, y = df_wordFreq.shape
    top_terms_list = []
    for i in range(math.ceil(x/3)):
    # for i in range(100):
        try:
            new_df = df[df['only_alphabets'].str.contains(df_wordFreq['term'].iloc[i])]#if word appears in review, create a dataframe with each row being the word occurring in a different review
            pos_first_df = new_df.sort_values(by='stars', ascending=False)#rank the dataframe with most positive reviews first
            if pos_first_df['text'].iloc[0] not in top_terms_list:#get the highest star rating review
                top_terms_list.append(pos_first_df['text'].iloc[0])
#                 top_terms_list.append(pos_first_df['text'].iloc[1])
        except IndexError as e:
            pass
    worst_terms_list = [] 
    for i in reversed(range(math.ceil(x/3), x)):
    # for i in range(-50,0):
        try:
            new_df = df[df['only_alphabets'].str.contains(df_wordFreq['term'].iloc[i])]#if word appears in review, create a dataframe with each row being the word occurring in a different review
            neg_first_df = new_df.sort_values(by='stars', ascending=True)#rank the dataframe with worst reviews first
            if neg_first_df['text'].iloc[0] not in worst_terms_list:#get the lowest star rating review
                worst_terms_list.append(neg_first_df['text'].iloc[0])#prevent duplicates
#                 worst_terms_list.append(neg_first_df['text'].iloc[1])#prevent duplicates
        except IndexError as e:
            pass
    del [df]
    negative_list = []
    # for i in range(-30,0):#take the worst 30 terms
    for i in reversed(range(math.ceil(2*x/3), x)):
        for list_of_words in worst_terms_list:
            word_list = list_of_words.split(' ')
            for word in word_list:
                try: 
                    if df_wordFreq['term'].iloc[i] == word: #find word occurrence in original comma separated word list of reviews
                        try:
                            index = word_list.index(word)
                            string_from_phrases = ' '.join(word_list[max(0,index-2):min(index+4, len(word_list))])
                            negative_list.append(string_from_phrases)
                        except ValueError as e:
                            pass
                except IndexError as e:#if there are less than 30 words after stopword filtering, just get the first word and its occurrence in the original review
                    if df_wordFreq['term'].iloc[0] == word:
                        try:
                            index = word_list.index(word)
                            string_from_phrases = ' '.join(word_list[max(0,index-2):min(index+4, len(word_list))])
                            negative_list.append(string_from_phrases)
                        except ValueError as e:
                            pass
    negative_df = pd.DataFrame(negative_list)
    negative_df = negative_df.reset_index(drop=False)

    negative_df = negative_df.rename(columns={'index':'score', 0 : 'term'})
    negative_df = negative_df.drop_duplicates(subset='term')
    x,y = negative_df.shape#tuple unpacking to get the length of the dataframe
    if x < 0:
        # for i in range(-40,-30):
        for i in reversed(range(math.ceil(x/3), math.ceil(2*x/3))):
            for list_of_words in worst_terms_list:
                word_list = list_of_words.split(' ')
                for word in word_list:
                    try:
                        if df_wordFreq['term'].iloc[i] == word:
                            try:
                                index = word_list.index(word)
                                string_from_phrases = ','.join(word_list[max(0,index-2):min(index+4, len(word_list))])
                                negative_list.append(string_from_phrases)
                            except ValueError as e:
                                pass
                    except IndexError as e:
                        if df_wordFreq['term'].iloc[0] == word:
                            try:
                                index = word_list.index(word)
                                string_from_phrases = ','.join(word_list[max(0,index-2):min(index+4, len(word_list))])
                                negative_list.append(string_from_phrases)
                            except ValueError as e:
                                pass
    negative_df_addon = pd.DataFrame(negative_list)
    negative_df_addon = negative_df_addon.reset_index(drop=False)
    negative_df_addon = negative_df_addon.rename(columns={'index':'score', 0 : 'term'})
    negative_df = pd.concat([negative_df, negative_df_addon])
    negative_df['term'] = negative_df['term'].str.replace(',',' ')
    negative_df['term'] = negative_df['term'].str.replace('\u00a0',' ')

    positive_list = []
    for i in range(math.ceil(x/3)):
    # for i in range(0,30):
        for list_of_words in top_terms_list:
            word_list = list_of_words.split(' ')
            for word in word_list:
                try: 
                    if df_wordFreq['term'].iloc[i] == word:
                        try:
                            index = word_list.index(word)
                            string_from_phrases = ','.join(word_list[max(0,index-2):min(index+4, len(word_list))])
                            positive_list.append(string_from_phrases)
                        except ValueError as e:
                            pass
                except IndexError as e:
                    if df_wordFreq['term'].iloc[0] == word:
                        try:
                            index = word_list.index(word)
                            string_from_phrases = ','.join(word_list[max(0,index-2):min(index+4, len(word_list))])
                            positive_list.append(string_from_phrases)
                        except ValueError as e:
                            pass
    positive_df = pd.DataFrame(positive_list)
    positive_df = positive_df.reset_index(drop=False)
    positive_df = positive_df.rename(columns={'index':'score', 0 : 'term'})
    positive_df = positive_df.drop_duplicates(subset='term')
    x,y = positive_df.shape#tuple unpacking to get the length of the dataframe
    # for i in range(30,40):
    for i in range((math.ceil(x/3))+1, math.ceil(x/1.5)):
        for list_of_words in top_terms_list:
            word_list = list_of_words.split(' ')
            for word in word_list:
                try:
                    if df_wordFreq['term'].iloc[i] == word:
                        try:
                            index = word_list.index(word)
                            string_from_phrases = ','.join(word_list[max(0,index-2):min(index+4, len(word_list))])
                            positive_list.append(string_from_phrases)
                        except ValueError as e:
                            pass
                except IndexError as e:
                    if df_wordFreq['term'].iloc[0] == word:
                        try:
                            index = word_list.index(word)
                            string_from_phrases = ','.join(word_list[max(0,index-2):min(index+4, len(word_list))])
                            positive_list.append(string_from_phrases)
                        except ValueError as e:
                            pass
    del [df_wordFreq]
    positive_df_addon = pd.DataFrame(positive_list)
    positive_df_addon = positive_df_addon.reset_index(drop=False)
    positive_df_addon = positive_df_addon.rename(columns={'index':'score', 0 : 'term'})
    positive_df = pd.concat([positive_df, positive_df_addon])
    positive_df['term'] = positive_df['term'].str.replace(',',' ')
    negative_df['term'] = negative_df['term'].str.replace('\u00a0',' ')
    return positive_df.head(topk), negative_df.tail(topk)


def getDataViztype0(business_id):
    ''' Deleted on 2020-01-13
    # do web scraping 
    yelpScraperResult = yelpScraper(business_id)
    '''
    data = getLatestReviews(business_id, limit=200)
    if len(data)==0:
        return {}
    df_reviews = pd.DataFrame(data, columns=['date', 'text', 'stars'])
    del data
    df_reviews['date'] = pd.to_datetime(df_reviews['date'])

    # viztype0
    df_positive, df_negative = getPosNegLongPhrases(df_reviews)
    positive, negative = [], []
    if not df_positive.empty:
        positive = [{'term': row[0], 'score': row[1]} 
            for row in df_positive[['term', 'score']].values]
    if not df_negative.empty:
        negative = [{'term': row[0], 'score': row[1]} 
            for row in df_negative[['term', 'score']].values]
    viztype0 = {
        'positive': positive, 
        'negative': negative
    }
    del [df_positive, df_negative]
    
    results = {
    'viztype0': viztype0

    }

    return results

In [38]:
business_id = "jga_2HO_j4I7tSYf5cCEnQ"

getDataViztype0(business_id)


{'viztype0': {'positive': [{'term': 'recommend that you stop by!', 'score': 0},
   {'term': 'make sure you mesh this is', 'score': 1},
   {'term': 'is what you would expect from', 'score': 2},
   {'term': 'to adopt you can get to', 'score': 3},
   {'term': 'guests. \xa0If you like cats this', 'score': 5},
   {'term': 'sure that you respect the cats', 'score': 6},
   {'term': 'are everywhere you look. Bob Marley', 'score': 11},
   {'term': 'Some let you hold them and', 'score': 12},
   {'term': 'of toys you can play with', 'score': 13},
   {'term': "even if you don't have those", 'score': 18}],
  'negative': [{'term': 'and am counting down the days', 'score': 1922},
   {'term': 'the days til I can come', 'score': 1923},
   {'term': 'atmosphere and ambiance made it so', 'score': 1924},
   {'term': 'and sweet  explained the "terms and', 'score': 1925},
   {'term': 'a pretty decent sized space too.', 'score': 1926},
   {'term': 'pretty decent sized space too. The', 'score': 1927},
   {'ter

In [22]:
# df_reviews

In [23]:
df_positive, df_negative = getPosNegLongPhrases(df_reviews)

In [26]:
df_negative

,score,term
4937,4937,The employee informed me they're still
4938,4938,with an unhealthy amount of cats.
4939,4939,my camera roll with an unhealthy
4940,4940,fill my camera roll with an
4941,4941,me the opportunity to fill my
4942,4942,them. Also allowed gave me the
4943,4943,to shelter letting people get to
4944,4944,a nice alternative to shelter letting
4945,4945,play with/snuggle/give treats is a bargain!!!
4946,4946,but am hoping to soon! The


In [27]:
positive = [{'term': row[0], 'score': row[1]} 
            for row in df_positive[['term', 'score']].values]
positive

[{'term': 'need to update their hours on', 'score': 0},
 {'term': 'website. My daughter has been talking', 'score': 9},
 {'term': 'has been talking about coming here', 'score': 18},
 {'term': 'open at noon and not the', 'score': 27},
 {'term': 'they have posted on their website.\n\nSorry', 'score': 36},
 {'term': 'for the smudged face prints on', 'score': 45},
 {'term': 'smudged face prints on your windows....', 'score': 54},
 {'term': 'There was no cafe. No coffee', 'score': 63},
 {'term': 'will allow no more than 15', 'score': 67},
 {'term': 'ambience  look no further! LaGattara is', 'score': 68}]

In [28]:
viztype0

NameError: name 'viztype0' is not defined